# SOLVERS ONLINE

Importing the libraries:

In [1]:
import numpy as np
from collections import defaultdict

Implementing the maze generator (for detailed implementation see `tests/maze_generator.ipynb`):

In [2]:
class UnionFind:
    def __init__(self, n):
        self.n = n  # number of supernodes
        self.v = [i for i in range(n)]  # represents the parents (initially each disjoint set has a single vertex)

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]  # compression technique
            u = self.v[u]
            
        return u
    
    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)
        
        if root_u == root_v:
            return False  # union was not performed
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True  # union was not performed

# Class for the maze generator
# Inspired by a randomized Kruskal algorithm
class KruskalMazeGenerator:
    def __init__(self, N=100):
        self.N = N  # number os cells (default is a 10x10 grid)
        self.shape = int(N**0.5)
        
        self.walls = self.generate_walls()
        
    def new_maze(self):        
        # Copy the walls (edges) and create a set for each cell
        walls = self.walls.copy()
        forest = UnionFind(self.N)
        
        # Generate a maze (represented by an adjacency list)
        maze = [[] for _ in range(self.N)]
        while forest.n > 1:
            v, w = walls.pop(np.random.randint(0, len(walls)))
            
            if forest.union(v, w):
                maze[v].append(w)  # adding (v, w) in the maze
                maze[w].append(v)  # adding (w, v) in the maze
                
        return maze
                
    def actions(self, v):  # takes into account the dimensions of the grid
        actions = []

        if v % self.shape != 0:
            actions.append((v, v-1))
        if (v+1) % self.shape != 0:
            actions.append((v, v+1))
        if v >= self.shape:
            actions.append((v, v-self.shape))
        if v+self.shape < self.N:
            actions.append((v, v+self.shape))

        return actions  # possible actions in cell v

    def generate_walls(self):
        walls = []
        
        # Movement represents the existence or not of a wall
        for v in range(self.N):
            walls.extend(self.actions(v))

        return walls

Implementing maze logic:

In [3]:
# Class responsible for the maze
# Default is a 10x10 grid
class Maze:
    def __init__(self, N=100):  # creates a new grid representing a maze
        self.__N = N
        self.__shape = int(self.N**0.5)

        self.__grid = KruskalMazeGenerator(self.N).new_maze()

    # Checks for the existence of an edge between two vertices
    def __getitem__(self, move):
        if isinstance(move, tuple) and len(move) == 2:
            pre, pos = move

            if 0 <= pos <= self.N-1:
                return pre in self.grid[pos]
            else:
                return False

        return None

    def won(self, x, y):
        return x == self.shape-1 and y == self.shape-1

    @property
    def N(self):
        return self.__N

    @property
    def shape(self):
        return self.__shape

    @property
    def grid(self):
        return self.__grid

Implementing the class responsible for the game character:

In [4]:
# Starts at position (0, 0)
class Fiancee:
    def __init__(self):
        self.x = 0
        self.y = 0

    def move(self, move):
        if (move == 'u'):
            self.y -= 1
        elif (move == 'd'):
            self.y += 1
        elif (move == 'r'):
            self.x += 1
        elif (move == 'l'):
            self.x -= 1
            
    @property
    def xy(self):
        return self.x, self.y

Auxiliary function to print the maze and the intelligent agent:

In [5]:
# (x, y) represents the position of the agent
def print_maze(maze, x, y):
    shape = int(len(maze)**0.5)
    
    for row in range(shape):
        for col in range(shape-1):
            print('o', end='') if x == col and y == row else print('x', end='')
            
            print(' ', end='') if 10*row + col in maze[10*row + col + 1] else print('|', end='')
        
        print('o') if x == col+1 and y == row else print('x')

        if row != shape-1:
            for col in range(shape):
                print(' ', end=' ') if 10*row + col in maze[10*(row + 1) + col] else print('-', end=' ')

            print()
            
# Testing the function

maze = Maze()
print_maze(maze.grid, 0, 0)

o x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


Defining the constants:

In [6]:
# Search status
UNSUCCESS = -1
NOT_STARTED = 0
STARTED = 1
SUCCESS = 2

## Online DFS agent

Implementing the online dfs agent:

In [7]:
# Undo actions
undone = {'u' : 'd',
          'd' : 'u',
          'r' : 'l',
          'l' : 'r'}

In [8]:
class OnlineDFSAgent:
    def __init__(self):
        self.results = defaultdict(lambda: None)
        self.untried = defaultdict(list)
        self.unbacktracked = defaultdict(list)
        
        self.s = None
        self.a = None
        
        self.status = NOT_STARTED
        
    # The s_line argument is a percept that identifies the current state
    def step_online(self, maze, s_line):
        # Checking if it found a goal state
        if maze.won(*s_line):
            self.status = SUCCESS
        
        # Identifying search status
        if self.status == SUCCESS:
            print("Goal already found!")
            return None
        elif self.status == UNSUCCESS:
            print("Unsuccessful search!")
            return None
        elif self.status == NOT_STARTED:
            self.status = STARTED
        
        if s_line not in self.untried:
            self.untried[s_line] = self.actions(maze, s_line)
            
        if self.s and s_line != self.results[self.s, self.a]:
            self.results[self.s, self.a] = s_line
            self.results[s_line, undone[self.a]] = self.s
            
            self.unbacktracked[s_line].append(undone[self.a])
        
        if self.untried[s_line]:
            self.a = self.untried[s_line].pop()
        elif self.unbacktracked[s_line]:
            self.a = self.unbacktracked[s_line].pop()
        else:
            self.status = UNSUCCESS
            return None
            
        self.s = s_line
        
        return self.a
            
    def actions(self, maze, pos):
        place = pos[0] + maze.shape*pos[1]
        
        actions = []
        
        if maze[place, place - 1] and self.a != 'r':
            actions.append('l')
        if maze[place, place - maze.shape] and self.a != 'd':
            actions.append('u')
        if maze[place, place + 1] and self.a != 'l':
            actions.append('r')
        if maze[place, place + maze.shape] and self.a != 'u':
            actions.append('d')

        return actions

Testing the agent:

In [9]:
agent = OnlineDFSAgent()
fiancee = Fiancee()

steps = 0
while agent.status != SUCCESS and agent.status != UNSUCCESS:
    # Movement of agents
    action = agent.step_online(maze, fiancee.xy)
    fiancee.move(action)
    
    # Printing the route
    print_maze(maze.grid, fiancee.x, fiancee.y)
    print("\n")
    
    steps += 1

x x|x x|x x x x x|x
  - -   - -   -     
o x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x o|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x o x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x

  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x o|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x o x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|o x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x

In [10]:
print(f'Final position = {fiancee.xy}')
print(f'Number of steps = {steps}')

Final position = (9, 9)
Number of steps = 137


## LRTA* agent

Implementing the LRTA* agent:

In [11]:
class LRTAStar:
    def __init__(self):
        self.results = defaultdict(lambda: None)
        self.H = dict()
        
        self.s = None
        self.a = None
        
        self.status = NOT_STARTED
        
    # The s_line argument is a percept that identifies the current state
    def step_online(self, maze, s_line):
        # Checking if it found a goal state
        if maze.won(*s_line):
            self.status = SUCCESS
        
        # Identifying search status
        if self.status == SUCCESS:
            print("Goal already found!")
            return None
        elif self.status == UNSUCCESS:
            print("Unsuccessful search!")
            return None
        elif self.status == NOT_STARTED:
            self.status = STARTED
        
        if s_line not in self.H:
            self.H[s_line] = self.manhattan(maze, *s_line)
            
        if self.s:
            self.results[self.s, self.a] = s_line
            
            self.H[self.s] = min([self.LRTA_cost(maze, self.s, self.results[self.s, b])
                                  for b in self.actions(maze, self.s)])
            
        self.s = s_line
        self.a = min([b for b in self.actions(maze, s_line)],
                     key=lambda b: self.LRTA_cost(maze, s_line, self.results[s_line, b]))        
        
        return self.a
            
    def actions(self, maze, pos):
        place = pos[0] + maze.shape*pos[1]
        
        actions = []
        
        if maze[place, place - maze.shape]:
            actions.append('u')
        if maze[place, place + 1]:
            actions.append('r')
        if maze[place, place + maze.shape]:
            actions.append('d')
        if maze[place, place - 1]:
            actions.append('l')

        return actions
    
    def LRTA_cost(self, maze, s, s_line):
        return 1 + self.H[s_line] if s_line else self.manhattan(maze, *s)
    
    @staticmethod
    def manhattan(maze, x, y):
        return abs(x - (maze.shape-1)) + abs(y - (maze.shape-1))  # manhattan heuristic

Testing the agent:

In [12]:
agent = LRTAStar()
fiancee = Fiancee()

steps = 0
while agent.status != SUCCESS and agent.status != UNSUCCESS:
    # Movement of agents
    action = agent.step_online(maze, fiancee.xy)
    fiancee.move(action)
    
    # Printing the route
    print_maze(maze.grid, fiancee.x, fiancee.y)
    print("\n")
    
    steps += 1

x o|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


o x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
o x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x

x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|o x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|o|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - - 

x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x o x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x o|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - 

x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x o|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x o x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x

x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x o|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x o x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x o x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - - 

      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|o
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|o
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x

-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x o x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x o x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|o|x x x
      - - - -

      - - - -       
o|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
x|x|x|x x x|x x x|x
  -     - -   - -   
o|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -   - - 
x x|x x x|x x|x x|x
      -     - -     
x|x x|x x x x x|x x


x x|x x|x x x x x|x
  - -   - -   -     
x x|x|x|x|x x|x x x
-         - - -     
x x x x x x x|x|x|x
      - - - -       
o|x|x|x x x|x x x|x
  -     - -   - -   
x|x x x x x|x|x x|x
  - - - -       -   
x x x x|x|x x x x|x
  - -         - - - 
x x|x|x|x x|x|x x x
      - - - -   -   
x|x|x x|x|x x x x|x
  - -       -

In [13]:
print(f'Final position = {fiancee.xy}')
print(f'Number of steps = {steps}')

Final position = (9, 9)
Number of steps = 717
